### 핀테크 뉴스 수집

1. 핀테크 지원센터 홈페이지 → 알림마당 → 핀테크 정보 → 핀테크 뉴스에 접속
https://fintech.or.kr/web/board/boardContentsListPage.do?board_id=6&menu_id=6501&miv_pageNo=
2. 일자별로 있는 [11/18 핀테크 주요뉴스] 안에 있는 뉴스 제목과 원문 link 수집
3. 원문링크로 들어가서 뉴스 본문을 2025년 1월 2일까지 수집 후(약 18page) DB에 저장
4. 데이터 베이스에 저장된 뉴스를 데이터프레임으로 불러와서 텍스트 전처리, Wordcloud, LDA 토픽 모델링
5. 2025년 핀테크 뉴스에서 가장 많이 다룬 주제를 찾고 서비스 기획 아이디어 제시하기

In [1]:
import requests
import re
import os
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
url="https://fintech.or.kr/web/board/boardContentsList.do"
payload = {"mode":"W", "board_id":"6","contents_id": "0579e2d50ca4449f97f5c21e6acf3d1c"}
r = requests.post(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, 'lxml')
soup

https://fintech.or.kr/web/board/boardContentsList.do?mode=W&board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
200


<html><head><script language="javascript">
$(document).ready(function(){

	// 댓글수 노출여부 확인후 숨기기(미구현)
	
		$(".comment_cnt").hide();
	

	// 답변 노출여부 확인후 숨기기(미구현)
	
		$(".reply_status").hide();
	

	// 모바일 전용 페이지 숨기지
	//$(".mobile_list").hide();

	$(function() {

	});

	$("#searchtxt").keydown(function(key) {
		if (key.keyCode == 13) {
			search();
		}
	});
});
</script>
</head><body><div class="boardlist_top">
<ul class="box">
<li class="select">
<div class="optionbox">
<select id="searchkey" name="searchkey">
<option value="I">전체</option>
<option value="T">제목</option>
<option value="C">내용</option>
</select>
</div>
</li>
<li class="input">
<div class="inpbox"><input class="txt" id="searchtxt" name="searchtxt" placeholder="검색어 입력" title="검색어 입력" type="text" value=""/></div>
</li>
<li class="button">
<button class="btn3 bg_blue" onclick="search();" title="검색" type="button">검색</button>
<!-- <button type="button" class="btn3 bg_gray" title="재검색">재검색</button> -->
</li>
</ul>
</div>
<!--// list_t

In [3]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
board_id = "6"
page = 1

payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
r_list = requests.post(list_url, data=payload_list)
soup_list = bs(r_list.content, "lxml")

rows = soup_list.select("table tbody tr")

for row in rows:
    date_td = row.select_one("td.last")
    date = date_td.get_text(strip=True) if date_td else "날짜없음"
    
    title_tag = row.select_one("td.txtl a")
    if title_tag:
        title = title_tag.get_text(strip=True)
        contents_id = title_tag["href"].split("'")[1]  # javascript:contentsView('ID')
    else:
        title = "제목없음"
        contents_id = None
    
    print(date, "-", title, "-", contents_id)

2025-11-12 - [경기도] 미래콘텐츠 성과공유회(AXR)에 초대합니다. (11/20(목) ~ 21(금)) - bf1b44ca61d74074a455cf21e180451c
2025-11-12 - [우리금융그룹] 2026 디노랩 서울 7기/부산 2기/경남 3기 모집(~12/03) - d4ad01f29e27456bacd3bf33c0668232
2025-09-17 - [금융보안원] 「데이터허브」 이용 안내 - b4b0b17c318d45158645c81c1472e29a
2024-02-20 - [국무조정실] 규제개혁신문고 제도 안내 - 51cb0e37c4064f2a985436e2ae3378df
2022-12-19 - [기술보증기금] 핀테크 기업을 위한 주요 보증상품 안내 - 3c24d05040984c79a7cdfd2c921952eb
2022-12-19 - [신용보증기금] 스타트업 지원 업무 안내 - 25f4360d5b9847a996828099e8d52ade
2022-06-20 - [한국무역보험공사] K-Sure 수출컨설팅 사업 안내 - 9c16609a99ad4c55914ffa9a7f2d58b7
2021-12-03 - [한국예탁결제원] 예탁결제원 보유 데이터 활용 안내 - 6b14460b5adc480aa050c4c535af8b72
2021-04-07 - [웰컴저축은행] 마이데이터 서비스 제휴 솔루션 - d4cbbeb3f9b3444cbf1856b48f9f2926
2025-11-21 - [11/21 핀테크 주요뉴스] - edd05197dbf749118cd7d16918ab3cc4
2025-11-20 - [11/20 핀테크 주요뉴스] - 12c003b7478b4c359c558ecc1edcc342
2025-11-19 - [11/19 핀테크 주요뉴스] - e5bc59a8918247a6943cc881219e5bec
2025-11-18 - [11/18 핀테크 주요뉴스] - 0579e2d50ca4449f97f5c21e6acf3d1c
2025-11-17 - [11/17 핀테크 주요뉴스

In [11]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
view_url = "https://fintech.or.kr/web/board/boardContentsView.do"

board_id = "6"
page = 1

payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
r_list = requests.post(list_url, data=payload_list)
soup_list = bs(r_list.content, "lxml")

rows = soup_list.select("table tbody tr")

for row in rows:
    date_td = row.select_one("td.last")
    date = date_td.get_text(strip=True) if date_td else "날짜없음"
    
    title_tag = row.select_one("td.txtl a")
    if title_tag:
        title = title_tag.get_text(strip=True)
        contents_id = title_tag["href"].split("'")[1]  # javascript:contentsView('ID')

        view_link = f"{view_url}?board_id={board_id}&contents_id={contents_id}"
    else:
        title = "제목없음"
        view_link = None
    
    print(date, "-", title, "-", view_link)

2025-11-12 - [경기도] 미래콘텐츠 성과공유회(AXR)에 초대합니다. (11/20(목) ~ 21(금)) - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=bf1b44ca61d74074a455cf21e180451c
2025-11-12 - [우리금융그룹] 2026 디노랩 서울 7기/부산 2기/경남 3기 모집(~12/03) - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=d4ad01f29e27456bacd3bf33c0668232
2025-09-17 - [금융보안원] 「데이터허브」 이용 안내 - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=b4b0b17c318d45158645c81c1472e29a
2024-02-20 - [국무조정실] 규제개혁신문고 제도 안내 - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=51cb0e37c4064f2a985436e2ae3378df
2022-12-19 - [기술보증기금] 핀테크 기업을 위한 주요 보증상품 안내 - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=3c24d05040984c79a7cdfd2c921952eb
2022-12-19 - [신용보증기금] 스타트업 지원 업무 안내 - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=25f4360d5b9847a996828099e8d52ade
2022-06-20 - [한국무역보험공사] K-Sure 수출컨설팅 사업 안내 - https://fintech.

In [12]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
view_url = "https://fintech.or.kr/web/board/boardContentsView.do"

board_id = "6"
page = 1

payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
r_list = requests.post(list_url, data=payload_list)
soup_list = bs(r_list.content, "lxml")

rows = soup_list.select("table tbody tr")

pattern = re.compile(r"\[\d{1,2}/\d{1,2}\s*핀테크 주요뉴스\]") 

for row in rows:
    date_td = row.select_one("td.last")
    date = date_td.get_text(strip=True) if date_td else "날짜없음"
    
    title_tag = row.select_one("td.txtl a")
    if not title_tag:
        continue
    
    title = title_tag.get_text(strip=True)

    if not pattern.search(title):
        continue

    contents_id = title_tag["href"].split("'")[1]
    view_link = f"{view_url}?board_id={board_id}&contents_id={contents_id}"

    print(date, "-", title, "-", view_link)

2025-11-21 - [11/21 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=edd05197dbf749118cd7d16918ab3cc4
2025-11-20 - [11/20 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=12c003b7478b4c359c558ecc1edcc342
2025-11-19 - [11/19 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=e5bc59a8918247a6943cc881219e5bec
2025-11-18 - [11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
2025-11-17 - [11/17 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=bc4b472316354894a61a9d7515fd585a
2025-11-14 - [11/14 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=06baae1d74e84020897b429e5908b67d


In [13]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
view_url = "https://fintech.or.kr/web/board/boardContentsView.do"

board_id = "6"
target_date = "11/18"

payload = {"mode": "W", "board_id": board_id, "pageIndex": 1}
r_list = requests.post(list_url, data=payload)
soup_list = bs(r_list.content, "lxml")

rows = soup_list.select("table tbody tr")

pattern = re.compile(rf"\[{target_date}\s*핀테크 주요뉴스\]")

detail_url = None

for row in rows:
    title_tag = row.select_one("td.txtl a")
    if not title_tag:
        continue

    title = title_tag.get_text(strip=True)

    if not pattern.search(title):
        continue

    contents_id = title_tag["href"].split("'")[1]
    detail_url = f"{view_url}?board_id={board_id}&contents_id={contents_id}"
    break

if not detail_url:
    print(f"{target_date} 주요뉴스를 찾지 못함")
    exit()

r_detail = requests.get(detail_url)
soup_detail = bs(r_detail.content, "lxml")

news_list = []

links = soup_detail.select("td a[target='_blank']")

for a in links:
    news_title = a.get_text(strip=True)
    news_link = a["href"]

    if news_link.startswith("//"):
        news_link = "https:" + news_link

    news_list.append((news_title, news_link))


print(f"\n=== [11/18 핀테크 주요뉴스] 내부 기사 ===\n")

for t, link in news_list:
    print(f"- {t} : {link}")


=== [11/18 핀테크 주요뉴스] 내부 기사 ===

- [한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Academy’참여자 모집(~선착순 마감) : https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=fe915b15bcca4d11b46d83b70a41207d&menu_id=6300
- [한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생 모집(상시) : https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=e04c846efa67421db116e5533c0d8bd2&menu_id=6300
- [한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) 참여기업 모집 (~예산 소진 시까지) : https://fintech.or.kr/web/board/boardContentsView.do?board_id=1&contents_id=abc17a4ab12d4c79ae94d177994b4d57&menu_id=6100
- 150조 '국민성장펀드' 본격 가동…이억원 "금융권 역량 총동원" : https://n.news.naver.com/mnews/article/015/0005212305?sid=101
- 금감원, '포인트 자동사용 서비스' 모든 카드사로 확대 : https://n.news.naver.com/mnews/article/003/0013604464?sid=101
- 금감원, '모험자본 워크숍' 개최…“발행어음·기업금융 동일 본부 지양” : https://n.news.naver.com/mnews/article/011/0004556759?sid=101
- 금융위 “빚투, 엄격한 리스크 관리 필요… 신용거래 매일 점검” : https://n.news.naver.com/mnews/article/020/0003675129?s

In [4]:
def text_clean(text):
    # HTML 태그를 없애는 정규표현식
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    result = result.replace("  ", " ").replace("  ", " ").replace("  ", " ")
    return result

In [18]:
BASE_URL = "https://fintech.or.kr"
LIST_URL = "https://fintech.or.kr/web/board/boardContentsList.do"
BOARD_ID = "6"

def get_article_text(url):
    try:
        r = requests.get(url, timeout=10)
        soup = bs(r.text, "lxml")
    except:
        return "요청 실패"

    # 네이버 뉴스
    if "news.naver.com" in url:
        area = soup.select_one("#dic_area")
        return area.get_text("\n", strip=True) if area else "본문 없음"

    # 핀테크지원센터
    area = soup.select_one(".board_detail")
    if area:
        return area.get_text("\n", strip=True)

    # 기타 fallback
    article = soup.find("article") or soup.find("div")
    return article.get_text("\n", strip=True) if article else "본문 없음"

def get_daily_news(contents_id):
    url = f"{BASE_URL}/web/board/boardContentsView.do?contents_id={contents_id}&board_id={BOARD_ID}"
    res = requests.get(url)
    soup = bs(res.text, "lxml")

    news_list = []

    # 'a[target=_blank]' = 실제 뉴스 링크들
    items = soup.select("td a[target='_blank']")

    for a in items:
        link = a.get("href")
        if not link or not link.startswith("http"):
            continue

        title = a.get_text(strip=True)
        text = get_article_text(link)

        news_list.append({
            "inner_title": title,
            "article_text": text
        })

    return news_list

STOP_TITLE = "[1/2 핀테크 주요뉴스]"
final_rows = []
stop_flag = False

for page in range(1, 19):
    print(f"{page}/18 페이지 수집 중...")

    payload = {"mode": "W", "board_id": BOARD_ID, "pageIndex": page}
    r = requests.post(LIST_URL, data=payload)
    soup = bs(r.text, "lxml")

    rows = soup.select("table tbody tr")

    for row in rows:
        title_tag = row.select_one("td.txtl a")
        date_td = row.select_one("td.last")

        if not title_tag:
            continue

        title = title_tag.get_text(strip=True)
        date_text = date_td.get_text(strip=True) if date_td else "날짜없음"

        # 날짜 포함한 뉴스만 수집
        if not re.match(r"^\[\d{1,2}/\d{1,2}", title):
            continue

        contents_id = title_tag["href"].split("'")[1]

        # 종료 조건
        if title.startswith("[1/2"):
            print("'[1/2 핀테크 주요뉴스]' 발견 → 종료합니다.")
            stop_flag = True

        # 상세 페이지 내부 뉴스 가져오기
        daily_news = get_daily_news(contents_id)

        for item in daily_news:final_rows.append({"날짜": date_text,"일자제목": title,"뉴스제목": item["inner_title"],
                                                  "본문": item["article_text"]})

        if stop_flag:
            break

    if stop_flag:
        break

    time.sleep(0.2)


df = pd.DataFrame(final_rows)
print("\n=== DataFrame 미리보기 ===\n")
print(df.head(20))

df

1/18 페이지 수집 중...
2/18 페이지 수집 중...
3/18 페이지 수집 중...
4/18 페이지 수집 중...
5/18 페이지 수집 중...
6/18 페이지 수집 중...
7/18 페이지 수집 중...
8/18 페이지 수집 중...
9/18 페이지 수집 중...
10/18 페이지 수집 중...
11/18 페이지 수집 중...
12/18 페이지 수집 중...
13/18 페이지 수집 중...
14/18 페이지 수집 중...
15/18 페이지 수집 중...
16/18 페이지 수집 중...
17/18 페이지 수집 중...
18/18 페이지 수집 중...

=== DataFrame 미리보기 ===

            날짜              일자제목  \
0   2025-11-21  [11/21 핀테크 주요뉴스]   
1   2025-11-21  [11/21 핀테크 주요뉴스]   
2   2025-11-21  [11/21 핀테크 주요뉴스]   
3   2025-11-21  [11/21 핀테크 주요뉴스]   
4   2025-11-21  [11/21 핀테크 주요뉴스]   
5   2025-11-21  [11/21 핀테크 주요뉴스]   
6   2025-11-21  [11/21 핀테크 주요뉴스]   
7   2025-11-21  [11/21 핀테크 주요뉴스]   
8   2025-11-21  [11/21 핀테크 주요뉴스]   
9   2025-11-21  [11/21 핀테크 주요뉴스]   
10  2025-11-21  [11/21 핀테크 주요뉴스]   
11  2025-11-21  [11/21 핀테크 주요뉴스]   
12  2025-11-21  [11/21 핀테크 주요뉴스]   
13  2025-11-21  [11/21 핀테크 주요뉴스]   
14  2025-11-21  [11/21 핀테크 주요뉴스]   
15  2025-11-21  [11/21 핀테크 주요뉴스]   
16  2025-11-21  [11/21 핀테크 주요뉴스]   
17  2025-11-

,날짜,일자제목,뉴스제목,본문
0,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...,[모집] 2025 핀테크 전문가 과정 ‘FinBoost Academy’ 모듈 4/5...
1,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...
2,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,[모집] (온라인 과정) 핀테크를 통한 금융 AI 트렌드와 혁신 사례(~12.31....
3,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,[한국핀테크지원센터] 2025년 ‘핀테크기업 온라인 채용관’ (사람인 saramin...
4,2025-11-21,[11/21 핀테크 주요뉴스],"금감원, IMA·발행어음 증권사 임원 대상 간담회 개최… 모험자본·리스크·소비자보호 당부",[연합뉴스]\n금융감독원이 한국투자증권·미래에셋증권·키움증권의 종합투자계좌(IMA)...
...,...,...,...,...
5989,2025-11-14,[11/14 핀테크 주요뉴스],[금융보안원] 「데이터허브」 기업 무료 이용 안내(상시),[금융보안원] 「데이터허브」 이용 안내\n작성일 : 2025-09-17\n[붙임1]...
5990,2025-11-14,[11/14 핀테크 주요뉴스],,로그인\n회원가입\nENG\n로그인\n회원가입\n금융규제\n샌드박스\n코리아\n핀테...
5991,2025-11-14,[11/14 핀테크 주요뉴스],,금융규제 샌드박스\n로 금융혁신에 더 가까이 가겠습니다.\n이전\n다음\n재생\n1...
5992,2025-11-14,[11/14 핀테크 주요뉴스],,


In [20]:
df['본문'] = df['본문'].apply(text_clean)

In [21]:
df

,날짜,일자제목,뉴스제목,본문
0,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...,모집 2025 핀테크 전문가 과정 FinBoost Academy 모듈 4 5 6 ...
1,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,한국핀테크산업협회 네트워킹 행사 핀테크 커넥팅 데이 2025 개최 11 26 신청...
2,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,모집 온라인 과정 핀테크를 통한 금융 AI 트렌드와 혁신 사례 12 31 상시모집...
3,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,한국핀테크지원센터 2025년 핀테크기업 온라인 채용관 사람인 saramin 참여 ...
4,2025-11-21,[11/21 핀테크 주요뉴스],"금감원, IMA·발행어음 증권사 임원 대상 간담회 개최… 모험자본·리스크·소비자보호 당부",연합뉴스 금융감독원이 한국투자증권 미래에셋증권 키움증권의 종합투자계좌 IMA 발행...
...,...,...,...,...
5989,2025-11-14,[11/14 핀테크 주요뉴스],[금융보안원] 「데이터허브」 기업 무료 이용 안내(상시),금융보안원 데이터허브 이용 안내 작성일 2025 09 17 붙임1 금융보안원 데이...
5990,2025-11-14,[11/14 핀테크 주요뉴스],,로그인 회원가입 ENG 로그인 회원가입 금융규제 샌드박스 코리아 핀테크 위크 D 테...
5991,2025-11-14,[11/14 핀테크 주요뉴스],,금융규제 샌드박스 로 금융혁신에 더 가까이 가겠습니다 이전 다음 재생 1 혁신금융서...
5992,2025-11-14,[11/14 핀테크 주요뉴스],,


In [ ]:
BASE_URL = "https://fintech.or.kr"
LIST_URL = "https://fintech.or.kr/web/board/boardContentsList.do"
BOARD_ID = "6"

def text_clean(text):
    # HTML 태그 제거
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외 문자 제거
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    # 연속 공백 1개로 변환
    result = re.sub(r"\s+", " ", result)
    return result.strip()

def get_article_text(url):
    try:
        r = requests.get(url, timeout=10)
        soup = bs(r.text, "lxml")
    except:
        return "요청 실패"

    # 네이버 뉴스
    if "news.naver.com" in url:
        area = soup.select_one("#dic_area")
        return area.get_text("\n", strip=True) if area else "본문 없음"

    # 핀테크지원센터
    area = soup.select_one(".board_detail")
    if area:
        return area.get_text("\n", strip=True)

    # 기타 fallback
    article = soup.find("article") or soup.find("div")
    return article.get_text("\n", strip=True) if article else "본문 없음"

def get_daily_news(contents_id):
    url = f"{BASE_URL}/web/board/boardContentsView.do?contents_id={contents_id}&board_id={BOARD_ID}"
    res = requests.get(url)
    soup = bs(res.text, "lxml")

    news_list = []
    items = soup.select("td a[target='_blank']")

    for a in items:
        link = a.get("href")
        if not link or not link.startswith("http"):
            continue

        title = a.get_text(strip=True)
        text = get_article_text(link)
        text = text_clean(text)  

 
        if not title or not text or text == "본문 없음":
            continue

        news_list.append({
            "inner_title": title,
            "article_text": text
        })

    return news_list

final_rows = []
stop_flag = False

for page in range(1, 20):
    print(f"{page}/19 페이지 수집 중...")
    payload = {"mode": "W", "board_id": BOARD_ID, "pageIndex": page}
    r = requests.post(LIST_URL, data=payload)
    soup = bs(r.text, "lxml")

    rows = soup.select("table tbody tr")

    for row in rows:
        title_tag = row.select_one("td.txtl a")
        date_td = row.select_one("td.last")

        if not title_tag:
            continue

        title = title_tag.get_text(strip=True)
        date_text = date_td.get_text(strip=True) if date_td else "날짜없음"

        if not re.match(r"^\[\d{1,2}/\d{1,2}", title):
            continue

        contents_id = title_tag["href"].split("'")[1]

        # 종료 조건: [1/2 ...] 뉴스
        if re.match(r"\[1/2\s", title):
            print("'[1/2 핀테크 주요뉴스]' 발견")

        # 상세 뉴스 가져오기
        daily_news = get_daily_news(contents_id)

        for item in daily_news:
            final_rows.append({"날짜": date_text,"일자제목": title,"뉴스제목": item["inner_title"],"본문": item["article_text"]})

        if stop_flag:
            break

    if stop_flag:
        break

    time.sleep(0.2)

df = pd.DataFrame(final_rows)

df = df[df['뉴스제목'].notna() & df['뉴스제목'].str.strip().astype(bool)]

print("\n=== DataFrame 미리보기 ===\n")
print(df.head(20))

1/19 페이지 수집 중...
2/19 페이지 수집 중...
3/19 페이지 수집 중...
4/19 페이지 수집 중...


In [7]:
# final_rows = []

# session = requests.Session()
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
#     "Referer": LIST_URL
# }

# for page in range(1, 21):
#     print(f"{page}/ 20 페이지 수집 중...")
    
#     payload = {"mode": "W", "board_id": BOARD_ID, "pageIndex": page}
#     r = session.post(LIST_URL, data=payload, headers=headers)
#     soup = bs(r.text, "lxml")

#     rows = soup.select("table tbody tr")
#     if not rows:  
#         print(f"{page} 페이지 데이터 없음 → 다음 페이지")
#         continue

#     for row in rows:
#         title_tag = row.select_one("td.txtl a")
#         date_td = row.select_one("td.last")

#         if not title_tag:
#             continue

#         title = title_tag.get_text(strip=True)
#         date_text = date_td.get_text(strip=True) if date_td else "날짜없음"

#         if not re.match(r"^\[\d{1,2}/\d{1,2}", title):
#             continue

#         contents_id = title_tag["href"].split("'")[1]

#         daily_news = get_daily_news(contents_id)

#         for item in daily_news:
#             final_rows.append({"날짜": date_text,"일자제목": title,"뉴스제목": item["inner_title"],"본문": item["article_text"]})

#     time.sleep(0.2)  

# df = pd.DataFrame(final_rows)
# df = df[df['뉴스제목'].notna() & df['뉴스제목'].str.strip().astype(bool)]

# print("\n=== DataFrame 미리보기 ===\n")
# print(df.head(20))

In [6]:
BASE_URL = "https://fintech.or.kr"
LIST_URL = "https://fintech.or.kr/web/board/boardContentsList.do"
BOARD_ID = "6"

final_rows = []

session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": LIST_URL
}

for page in range(1, 21):
    print(f"{page}/20 페이지 수집 중...")
    
    payload = {"mode": "W","board_id": BOARD_ID,"miv_pageNo": page, "LISTOP": "W"}
    
    r = session.post(LIST_URL, data=payload, headers=headers)
    soup = bs(r.text, "lxml")

    rows = soup.select("table tbody tr")
    if not rows:  
        print(f"{page} 페이지 데이터 없음 → 다음 페이지")
        continue

    for row in rows:
        title_tag = row.select_one("td.txtl a")
        date_td = row.select_one("td.last")

        if not title_tag:
            continue

        title = title_tag.get_text(strip=True)
        date_text = date_td.get_text(strip=True) if date_td else "날짜없음"

        if not re.match(r"^\[\d{1,2}/\d{1,2}", title):
            continue

        contents_id = title_tag["href"].split("'")[1]

        daily_news = get_daily_news(contents_id)

        for item in daily_news:
            final_rows.append({"날짜": date_text,"일자제목": title,"뉴스제목": item["inner_title"],"본문": item["article_text"]})

    time.sleep(0.2)  

df = pd.DataFrame(final_rows)
df = df[df['뉴스제목'].notna() & df['뉴스제목'].str.strip().astype(bool)]

print("\n=== DataFrame 미리보기 ===\n")
print(df.head(20))

1/20 페이지 수집 중...


NameError: name 'get_daily_news' is not defined

In [25]:
df

,날짜,일자제목,뉴스제목,본문
0,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...,모집 2025 핀테크 전문가 과정 FinBoost Academy 모듈 4 5 6 참...
1,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,한국핀테크산업협회 네트워킹 행사 핀테크 커넥팅 데이 2025 개최 11 26 신청 ...
2,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,모집 온라인 과정 핀테크를 통한 금융 AI 트렌드와 혁신 사례 12 31 상시모집 ...
3,2025-11-21,[11/21 핀테크 주요뉴스],[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,한국핀테크지원센터 2025년 핀테크기업 온라인 채용관 사람인 saramin 참여 기...
4,2025-11-21,[11/21 핀테크 주요뉴스],"금감원, IMA·발행어음 증권사 임원 대상 간담회 개최… 모험자본·리스크·소비자보호 당부",연합뉴스 금융감독원이 한국투자증권 미래에셋증권 키움증권의 종합투자계좌 IMA 발행어...
...,...,...,...,...
5866,2025-11-14,[11/14 핀테크 주요뉴스],"'빅테크 규제 응징' 트럼프 경고에도…EU, 구글 공식 조사 착수",구글이 스팸 방지 정책 을 내세워 자사 검색 결과에서 언론사와 뉴스 매체를 부당하게...
5867,2025-11-14,[11/14 핀테크 주요뉴스],[우리금융그룹] 2026 디노랩 서울 7기/부산 2기/경남 3기 참여기업 모집(~1...,우리금융그룹 2026 디노랩 서울 7기 부산 2기 경남 3기 모집 12 03 작성일...
5868,2025-11-14,[11/14 핀테크 주요뉴스],[경기도] 미래콘텐츠 성과공유회(AXR)에 초대합니다. (11.20.~21.),경기도 미래콘텐츠 성과공유회 AXR 에 초대합니다 11 20 목 21 금 작성일 2...
5869,2025-11-14,[11/14 핀테크 주요뉴스],[IBK기업은행]2026년 상반기 IBK 창공(대전) 9기 기업 모집(~11.14....,IBK기업은행 2026년 상반기 IBK 창공 대전 9기 기업 모집 11 14 14 ...


In [26]:
import pandas as pd
from dbio3 import to_db, load_data

dbname = "fintech_news"
table_name = "news"

to_db(dbname, table_name, df)
print("DB에 저장 완료!")

df_from_db = load_data(dbname, table_name)
print("\nDB에서 불러온 데이터 미리보기:")
print(df_from_db.head())

fintech_news 데이터베이스 확인/생성 완료
DB에 저장 완료!
fintech_news 데이터베이스 확인/생성 완료

DB에서 불러온 데이터 미리보기:
           날짜              일자제목  \
0  2025-11-21  [11/21 핀테크 주요뉴스]   
1  2025-11-21  [11/21 핀테크 주요뉴스]   
2  2025-11-21  [11/21 핀테크 주요뉴스]   
3  2025-11-21  [11/21 핀테크 주요뉴스]   
4  2025-11-21  [11/21 핀테크 주요뉴스]   

                                                뉴스제목  \
0  [한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...   
1  [한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...   
2  [한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...   
3  [한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...   
4  금감원, IMA·발행어음 증권사 임원 대상 간담회 개최… 모험자본·리스크·소비자보호 당부   

                                                  본문  
0  모집 2025 핀테크 전문가 과정 FinBoost Academy 모듈 4 5 6 참...  
1  한국핀테크산업협회 네트워킹 행사 핀테크 커넥팅 데이 2025 개최 11 26 신청 ...  
2  모집 온라인 과정 핀테크를 통한 금융 AI 트렌드와 혁신 사례 12 31 상시모집 ...  
3  한국핀테크지원센터 2025년 핀테크기업 온라인 채용관 사람인 saramin 참여 기...  
4  연합뉴스 금융감독원이 한국투자증권 미래에셋증권 키움증권의 종합투자계좌 IMA 발행어...  


In [28]:
#df.to_csv("./data/핀테크뉴스수집.csv", index=False, encoding="utf-8-sig")